# Binary Search Trees

Last week we looked at dynamic arrays and linked lists, two data structures that could be used to store data as elements in a sequence. We used these to implement *lists*, a data type where one can easily add and remove elements. We also compared how the different data structures lead to different performances of the list data type.

This week, we will turn to a new data structure: the *binary search tree*. The binary search tree is quite a bit different from linked lists and dynamic arrays, and we cannot use this data structure to implement the same type of lists, as the data structure won't support all the same operations. We will still find it useful to compare these three different data structures, to understand how they differ and how they are suited to different problems.

## The  `search` operation

Recall that in the previous lecture, we compared the performance of a linked list and a dynamic array on a few different operations, namely inserting, and indexing elements. We found that these structures had different computational cost from each other for the different operations.

Now we want to add a new type of operation we can perform on a data structure: the `search` operation. Where an indexing operation finds an element in the sequence based on its postion or index, the *search* operation finds an element based on its *value*.

We call this operation *search*, because to implement it we have to search for the correct value. If and when we find it, we can do a few things, we could for example return the index of the element. For example, Python lists have a method called `.index()` that does exactly this:

In [2]:
example = [4, 3, 2, 4, 0, 1, 7, 0, 8, 0, 3, 4]
print(example.index(0))

4


The result is 4, because this is the first index in the list with an element with the value 0. Note that this method only returns the first element with the correct value. We could instead return a list of all indices with the given value if we wanted to. If we try to use `index` on a value that does not exist, we get an `IndexError`.

Another possibility would be to implement a method called something like `contains`, which would return a boolean if the list contains the element or not. In Python, the natural way to check if a container object contains a given element is to do the following:

In [4]:
print(6 in example)

False


However, this is actually a contains method being called in the background. In fact, it is the special method called `__contains__` that does this.

### Searching on Linked Lists and Dynamic Arrays

Let us implement the `contains` method for both of our data structures from last week:

#### Dynamic Array Search
***
```C++
bool contains(int value) {
    for (int i=0; i<size; i++) {
        if (data[i] == value) {
            return true;
        }
    return false;
}
```
***
Here we assume that the storage array is called `data` and that the number of variables is stored in a `size` variable.


This function is fairly straight forward. It simply iterates through the array and compare each element to the value we are looking for. If we find a match, we instantly return true. If we have iterated through all values, and still haven't found it, then we finally return false.

What is the cost of this operation? Well, the worst case is that the element is at the end of the array, or that it doesn't exist, in which case we have to iterate through all the $n$ elements. Thus, the worst case of our operation is $\mathcal{O}(n)$. What about the average case scenario? Well, assuming the array has a single match, and it is randomly distributed within the array, we need to check about half the array on average (sometimes it is found early, sometimes it is further back). Checking $n/2$ elements is still $\mathcal{O}(n)$, if the array doubles, searching will also take twice as long. 

#### Linked List Search

***
```C++
bool contains(int value) {
    Node* current = head;
    
    while (current != nullptr) {
        if (current->value == value) {
            return true;
        }
        current = current->next
    }
    return false;
}
```
***
The process is just like for the array, we iterate through the linked list, checking each element if it is the correct one. Just like for the array, we have to iterate through the entire linked list to find our element (or no element) in the worst case. So just like the array search, we have a complexity of $\mathcal{O}(n)$ for the best and average cases.

### Can we do better than  $\mathcal{O}(n)$ for searching arrays and linked lists?

We found that searching either of our list data structures for a given value scales linearily: $\mathcal{O}(n)$. This means that searching these structures takes a long time if they get big. Can we do better?

No, we cannot. This shouldn't be terribly surprising. The fact is that the *values* of the elements/nodes in these containers doesn't impact where in the structure they are stored. Therefore, checking one element of the list tells us nothing about what the other elements contain, and we therefore have to check all the elements in the list to be sure to find our match, or to rule out that a match exists.

However, we now turn to a new data structure, called a *binary search tree*. As the name implies, this structure is easier to sort through, than our list structures. This isn't the only motivation for using binary search trees, but it is one operation where it will outperform our list structures.

## Binary Trees

After having seen and understood linked lists, understanding the basis of a binary tree is actually fairly easy. A linked list consists of *nodes* with a given data value, and a pointer to the next node in the list. A binary tree also consists of nodes, with each node having some data value. The binary tree node however has *two* "next" pointers. To separate them, we call them *left* and *right*. 

Because each node has two pointers, a single node can point to two nodes, each of which can point to two new nodes for a total of four. Those four can point to eight new ones and so on. We get a *branching* data structure, which is why we call it a *tree*. Because each node has two pointers, each branch splits into two, which is why it is a *binary* tree.

Let us draw a binary tree structure:

<img src="fig/linked_list_vs_bst.png" width=450>
<center><b>Figure 1:</b> (Top) A linked list structure, each node only points to the next node in the chain. The class keeps track of the head node, and the tail node. The tail node terminates by pointing at a nullptr. (Bottom)  A binary tree structure. Each new nodes points to two new nodes. The class only keeps track of the first node, called the root node. We call the next nodes in the tree child nodes. The nodes at the end of the tree that has no children are called the leaf nodes or terminal nodes, these only point at nullptrs. Compared to the linked lists, we get a lot more null pointers. </center>

Looking at the figure, we see that a binary tree class will typically keep track of the first node, called the *root* node. At the other end, we get the *leaf nodes*. It may be confusing that we draw the root node at the top, and the leaf nodes on the bottom. It's conventional to draw the trees "upside down" like this, probably because the "root" is the starting element, and we usually read documents from top to bottom.

We will only cover binary trees in IN1910, but we could in theory also create a tree structure where each node points to *three* new nodes, which would be a *tertiary* tree, or four nodes each, which would be a *quad tree* and so on.

The rest of this lecture will be on implementing a binary search tree class in C++, and analyzing its performance and behavior. If you are reading these notes on your own, I highly recommend you stop after each section and implement it on your own and verify things are working before moving on.

### Implementing a Binary Search Tree class

We want to implement a version of a binary search tree (BST). We start of with our node-struct. Just like for linked lists, the *nodes* can contain any kind of data, strings, integers, floats, or more complex data types. For our example we will store integers, just like earlier examples. In this case however, we call this the *key* of the node. We will explain why a bit later. The key is still just the value the node contains.

The struct can be defined as follows:
***
```C++
struct Node {
    int key;
    Node* left;
    Node* right;
    
    // Constructor
    Node(int value) {
        key = value;
        left = nullptr;
        right = nullptr;
    }
};
```
***
Our node contains three attributes, the `key`, and the two pointers `left` and `right`. The constructor takes the value of the node in, and starts the two pointers pointing to `nullptr`.

Next, we define our BST class, with its constructor:
***
```C++
class BinarySearchTree {
    private:
        Node *root;
        int count;
    
    public:
        BinarySearchTree() {
            root = nullptr;
            count = 0;
        }
    
        int length() {
            return count;
        }
}; 
```
***
Note that unlike our linked list class, our BST class only keeps track of the *root* node of our tree. As the tree grows, there will be more and more leaf nodes, and so it infeasible, or at least unpractical, to keep track of all the leaves. The BST class constructor starts the root pointing to null, meaning we have an empty tree.


#### Adding elements to our tree

We now want to start adding elements to our tree. For our lists, we started with an `append` method, but now, "appending" doesn't really make sense, as there is no single "end" of the tree to append to. Instead we call this operations "inserting" into the tree. For our lists, *inserting* was an operation where we specified the index, i.e. postion, of what we inserted. For our BST however, it is the search property which defines where the new element should be placed, and so our BST insert does not take any index, it simply takes a value to be inserted into the tree:

For an empty tree, this is trivial:
***
```C++
void insert(int value) {
    count++;
    if (root == nullptr) {
        root = new Node(value);
    }
}
```
***

But what when the tree *isn't* empty? Say we first add an element, and then want to add a second. The main question we face is then, what branch do we use? The left or the right?

Here we can choose different rules for our tree, and depending on the rule we choose, the tree data structure will behave differently. Because we want our tree to be easily searchable, we need to find some rule we can consistently follow to place nodes were they can be easily found. This is what turns our tree from a general binary tree to a binary *search* tree. 

The rule we choose is called the *search property* of the tree, and for a BST it is generally:
> Starting at the root, if our new node is less or equal to the current node, follow the left branch, if it is greater, follow the right branch. Repeat this process untill you find room to add your element.

Take for example the tree shown in Figure 1, if we want to add a new element with a value of "5", where does it end up? We start at the root, which is 2, so we go right (because 5 is larger than 2), the next node we meet is 6, so we move left (because 5 is smaller than 6), next we see 4, so we move right (because 5 is larger than 4). At this point we encounter a null pointer, as the "4" node does not have a child at its right pointer. Encountering a null pointer means we have space to add our new node here, and we therefore enter our "5" at this point. Our tree has grown, and "4" is no longer a leaf node, but has "5" as a child, which in turn, is a leaf node. Drawing out this example can help.

<img src="fig/bst_insert.png" width=200>
<center><b>Figure 2:</b> Inserting the value "5" into the example BST shown in Figure 1. It will become the right child of the "4"-node. </center>

When inserting elements into a tree, we must always follow the search property. Because we always follow the rule when inserting, we can guarantee that this property is always true for our tree in general. Put another way, we know that *all* elements down the left branch from a node will be smaller or equal than the node's key, and *all* elements down the right branch from a node will be larger than the node's key.

Going down a branch in the tree actually defines a *subtree*, that is a full binary search tree of its own. Just draw a tree for yourself if you are not convinced of this. This means that the binary search tree structure is a *recursive* structure, much in the same way you can define a *sublist* by slicing a list.

Even leaf nodes effectively point at subtrees. As they are pointing at null, they are effectively pointing at "nothing". But what is an empty tree, if not "nothing"?

<img src="fig/bst_subtrees.png" width=200>
<center><b>Figure 3:</b> In a binary search tree, a node points to two new nods that can be considered the roots of their own subtrees. The search property guarantees that <i>all</i> elements in the left subtree are smaller or equal than the root node, while <i>all</i> elements in the right subtree are larger.</center>

#### Implementing the insert opeartion

Inserting into an empty tree was close to trivial. But inserting into a tree that is not empty can be more tricky. For one, we need to respect the *search property* of the tree. Secondly, we need to keep going untill we find an empty spot. This means the code itself can become a bit messy with loops and tests.

When it comes to implementing BST methods, there are two major different ways of going about it. First we could try to implement an *iterative* method, i.e., using explicit loops untill we get to the end

##### Iterative Insert Method
***
```C++
void insert(int x) {
    count++;
    // Check for empty tree
    if (root == nullptr) {
        root = new Node(x);
        return;
    } 
    
    // Iterate to bottom of tree
    Node* parent = nullptr;
    Node* current = root;
    
    while (current != nullptr) {
        parent = current;
        if (x <= current->key) {
            current = current->left;
        } else
            current = current->right;
    }
    
    // Do actual insertion
    if (x <= parent->key) {
        parent->left = new Node(x);
    } else
        parent->right = new Node(x);
}         
```
***

Personally, I find this code becomes a bit messy. A different way to go about doing things, is to think *recursively*. Looking at Figure 3, we see that we check which subtree our new node should go in, and then call the *insert* method of that subtree. Wi will then recursively call the `insert` method of smaller and smaller subtrees untill we reach an empty tree, in which case we know how to add the node.

##### Recursive insert method
***
```C++
public:
    void insert(int x) {
        root = insert(root, x);
        count++;
    }

private:
    Node* insert(Node* subroot, int x) {
        // If subtree is empty
        if (subroot == nullptr) {
            return new Node(x);
        }
        
        // Subtree isn't empty, have to go deeper (recursively)
        if (x <= subroot->key) {
            subroot->left = insert(subroot->left, x);
        } else {
            subroot->right = insert(subroot->right, x);
        }
        return subroot;
    }
```
***

Note especially that we create two methods now. One is the public `insert` method that the user uses on the data type itself, this simply inserts an element into the whole tree. The other, is a helper function we just use to implement things, we therefore set it to private. The helper function is called on the node of a given sub-tree, we therefore call it the *subroot*.

Now is a good time to implement both of these methods yourself, actively thinking about what is happening in the code and how it works. If you do not have a lot of experience with recursive algorithms, then you might prefer the iterative method. I, however, personally think the recursive method comes out fairly elegant, compared to the iterative.

Just like this insertion method, we can implement pretty much *any* operation on our BST in an iterative, or recursive method. For the rest of these lecture notes, we will only show example implementation of the recursive manner. This is partly because I have a personal preference for that way of doing things, but mostly because it is what I think you might find the most challenging to do yourself. It could be a good exercise to go through and implement the same methods for yourself, but this time doing it iteratively instead.

### Printing our tree

Now that we have written an insertion method, we can start setting up trees by adding more and more numbers. But how can we control that everything is working correctly? We need to find some way to print out our tree. However, unlike the linked list or the array, there is no obvious way to iterate over the tree. Let us find a way to do this.

The way we iterate over the tree is again most easily understood by thinking recursively. To print out our tree, we start at the root and follow three easy steps:
1. Print out the left subtree
2. Print out the root node itself
3. Print out the right subtree 
This is often referred to as *traversing* the tree, rather than iterating over it.

As before, we keep recursively moving down untill we reach a leaf node, with no children, and in this case, we simply print the node and return. As before, we print the whole tree by calling the private `print`-method on the main root. Let us look at some code.

```C++
public:
    void print() {
        if (root == nullptr) {
            cout << "Empty tree." << endl;
        } else {
            print(root);
        }
}

private:
    void print(Node* subroot) {
        if (subroot == nullptr) {
            return;
        }
        
        print(subroot->left);
        cout << subroot->key << endl;
        print(subroot->right);
    }
```
Except for the special handling of empty trees, and empty subtrees, we see that the algorithm comes down to following the three steps we outlined above. Visiting (or printing) nodes in what we can call "left branch"-"root"-"right branch" order is only one possible way of traversing a binary tree.

Choosing to visit (print them out) in "left"-"root"-"right" order is only one possible way of traversing a binary tree. This is the most common way of traversing, it is known as *inorder traversal*. Other posibilities are printing them out in "root"-"left"-"right" order, which is called *preorder*, or "left"-"right"-"root" called *postorder* traversal.

<img src="fig/inorder_traversal.png" width=200>
<center><b>Figure 4:</b> Inorder traversal of a tree recursively handles all nodes in the left subtree, then the root itself, then the right subtree. Here the order the nodes are processed in are marked with blue numbers written just outside the node. It can be helpful to go through in-order traversal by hand on this example tree.</center>

Looking at the example in Figure 4, we can confirm that our newly-implemented printing method behaves in the manner we expect by testing:
```C++
BinarySearchTree tree;
tree.insert(3);
tree.insert(6);
tree.insert(1);
tree.insert(2);
tree.insert(4);
tree.insert(7);

tree.print()
```
Which outputs:
```
1
2
3
4
6
7
```
So we see, from both our by-hand example, and from actually testing, that the numbers we have inserting into our tree prints out the elements in sorted order! This isn't just a coincidence either, it is a side effect of doing in-order traversal. In fact, this is exactly why we call it *in-order* traversal, as the elements are printed/visited *in-order*. This is very different from both our list structures, in which printing our list object means printing out the elements in the order in which they were added to the list.

##### Sequences and insertion order

For our lists, appending elements puts them on the end of the list. And priting out the elements after appending a bunch of elements, prints them out in the same order they were appended in. We say that lists remember the *insertion order*.

While the order of insertion is also important for the structure of our binary search tree, it won't affect what is printed by in-order traversal of our tree, because this is in sorted order. In fact, it would be *impossible* to print out things in *insertion* order from a BST, because the structure doesn't store this information! 

As an example. Imagine making an array by appending the elements
* 2, 3, 1, 0
in order. Next create a new array by inserting
* 2, 1, 0, 3
These two arrays will be fundamentally different due to the insertion order. However, now do the same exercise, inserting the elements into a binary search tree instead, and you will find that both trees will have the exact same structure. Thus, these two trees cannot be differentiated on their input order. To verify, draw out the trees yourself and check this.

Note that it isn't generally true that two trees made from the same elements will be the same, if the order is shuffled about. For example, the first element inserted into a tree will become the root, no matter what. All we are trying to point out is that a tree does not remember the insertion order in the same manner as lists.

As the *order* of elements is a fundamental property of a sequence, a BST structure is *not* a sequence. This also means we cannot implement a list data type from a binary search tree.

##### Overloading the insert and constructor methods

In our test snippet above, we added a bunch of numbers one by one into our tree. This is not that efficient. To make testing and using the trees easier, we now overload our constructor and insert methods to be able to send in a bunch of numbers in one single call.
***
```C++
public:
    void insert(vector<int> numbers) {
        for (int n : numbers) {
            insert(n);
        }
    }
```
***
We can now call `insert` with a single number: `insert(4)`, or with a vector of numbers `insert({4, 2, 3, 1})`. With this done, we can also overload the constructor:
***
```C++
public:
    BinarySearchTree() {
        root = nullptr;
        count = 0;
    }
    
    BinarySearchTree(vector<int> numbers) {
        root = nullptr;
        count = 0;
        insert(numbers);
    }
```
***
So we can now create an empty tree with `BinarySearchTree tree()`, or create one with some initial content with `BinarySearchTree({4, 3, 1, 2})`. We can now easily create a new test of our in-order printing:
```C++
BinarySearchTree tree({7, 4, 1, 3, 5, 6, 2, 9, 0, 8});   
tree.print();
```
Which outputs the numbers 0 through 9, in order.